In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import os,sys,inspect

current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir)

import ipywidgets as widgets
from IPython.display import display

import yfinance as yf
from pandas_datareader import data as web

In [2]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [18]:
etf_list = ["DNL",
  "SPHQ",
  "VIG",
  "OGIG",
  "PTH",
  "PTF",
  "KBWY",
  "KBWD",
  "TADS",
  "ALTY",
  "SDIV",
  "DIV",
  "NUSI",
  "WBIY",
  "HEWC",
  "POTX",
  "IPFF",
  "ZCAN",
  "FLCA",
  "BBCA",
]

In [20]:
data = yf.download(etf_list, period="ytd")

[*********************100%***********************]  20 of 20 completed


In [10]:
data

Adj Close                                                        \
                ALTY      DIV      DNL     KBWD     KBWY     NUSI     OGIG   
Date                                                                         
2021-01-04  11.55000 16.85000 76.98000 16.27000 20.32000 27.42000 53.81000   
2021-01-05  11.65400 16.96000 77.77000 16.40000 20.37000 27.50000 55.15000   
2021-01-06  11.81000 17.35000 77.64000 16.93000 20.78000 27.34500 53.60000   
2021-01-07  11.78000 17.35000 78.17000 16.98000 20.58000 27.65000 54.96000   
2021-01-08  11.84000 17.31000 79.15000 17.00000 20.62000 27.75500 56.18000   

                                         ...  Volume                         \
                 PTF       PTH     SDIV  ...    KBWY    NUSI    OGIG    PTF   
Date                                     ...                                  
2021-01-04 138.61000 158.96001 12.86000  ...  108800  140200  308300  68800   
2021-01-05 140.11000 159.41000 12.96000  ...   69900   93200  188000  32200   
2021-01-06 137.49001 162.41000 13.10000  ...   80000   90700  260000  37200   
2021-01-07 144.44000 171.81000 13.11000  ...   89500   90400  206800  16900   
2021-01-08 144.71001 174.74001 13.19000  ...  104200  115500  209300  25500   

                                                          
               PTH    SDIV    SPHQ   TADS      VIG  WBIY  
Date                                                      
2021-01-04  158200  582500  380800   4600  2700900  7500  
2021-01-05   29700  389700  367200  48600  1376600  2900  
2021-01-06   29500  495900  562400   3000  1871300  5800  
2021-01-07   45800  387700  399200   5000  1415500  4700  
2021-01-08   46400  520600  260000    900  1705500  4900  

[5 rows x 84 columns]

In [21]:
from pypfopt import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [25]:
returns = data["Close"].pct_change().dropna()

In [40]:
dataframes = pd.DataFrame()
for stock in etf_list:
   dataframes[stock] = data['Adj Close'][stock]

In [41]:
mu = expected_returns.return_model(dataframes, method="capm_return")
S = risk_models.risk_matrix(dataframes, method="oracle_approximating")

In [42]:
ef = EfficientFrontier(mu, S)

In [43]:
raw_weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

In [44]:
ef.portfolio_performance(verbose=True)

Expected annual return: 3101.1%
Annual volatility: 11.6%
Sharpe Ratio: 266.10


(31.010669039420844, 0.11646293452369244, 266.0989881988274)

In [45]:
from pypfopt import DiscreteAllocation

da = DiscreteAllocation(cleaned_weights, dataframes.iloc[-1], total_portfolio_value=10000)
alloc, leftover = da.lp_portfolio()
print(f"Leftover: ${leftover:.2f}")
pd.Series(alloc).sort_index()

Leftover: $1.16


BBCA     27
DIV      16
FLCA     25
HEWC     23
IPFF     13
KBWD     13
NUSI      5
OGIG      3
POTX    198
PTF       9
PTH       3
SPHQ      8
TADS     12
VIG       1
WBIY     44
ZCAN     12
dtype: int64